In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch


In [2]:
data = pd.read_csv("clean_weather.csv")
data = data.ffill()


In [3]:
DEVICE=  torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
from torch.utils.data import DataLoader,Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length):
        self.data = data
        self.sequence_length = sequence_length
        
    def __len__(self):
        return len(self.data) - self.sequence_length
        
    def __getitem__(self, idx):
        # Get sequence of data
        sequence = self.data[idx:idx + self.sequence_length]
        return sequence

def transform(data,batch_size,sequence_len,device,dtype=torch.float32):
    
    data_tensor = torch.tensor(data,dtype=dtype,device=device)
    dataset = TimeSeriesDataset(data_tensor,sequence_len)
    
    dataloader = DataLoader(dataset,batch_size,shuffle=False)
    
    return dataloader
    

In [31]:
# Assuming 'date' is your date column and data is sorted by date
from sklearn.preprocessing import StandardScaler

predictors = ["tmax", "tmin", "rain"]
target = "tmax_tomorrow"

# Calculate split points
total_rows = len(data)
train_end = int(0.7 * total_rows)
valid_end = int(0.85 * total_rows)

# Split the data
X_train = data[predictors].iloc[:train_end]
y_train = data[target].iloc[:train_end]

X_valid = data[predictors].iloc[train_end:valid_end]
y_valid = data[target].iloc[train_end:valid_end]

X_test = data[predictors].iloc[valid_end:]
y_test = data[target].iloc[valid_end:]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

# Convert to a dataloader object with batch_size

X_train_scaled = np.array(X_train_scaled)
X_valid_scaled = np.array(X_valid_scaled)
X_test_scaled = np.array(X_test_scaled)

#We dont scale target data check on google for more info on why
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

In [32]:
x_train = transform(X_train_scaled,32,7,DEVICE)
x_valid = transform(X_train_scaled,32,7,DEVICE)
x_test = transform(X_test_scaled,32,7,DEVICE)
y_train = transform(y_train,32,7,DEVICE)
y_valid = transform(y_valid,32,7,DEVICE)